# Pileups at Centromeres

In [ ]:
%matplotlib inline
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
mpl.style.use('seaborn-white')
import multiprocess as mp
import numpy as np
import pandas as pd
import bioframe
import cooltools
import cooler

In [ ]:
#INPUTs

# Chromosome sizes file (chrom length)
chromsizesfile = 'PATH_TO/chrom.sizes.file'
# Sites of interest BED (chrom start end strand)
centromeresfile = 'PATH_TO/centromere.bed.file'
# Bin size
binsize = 1000
# Flank size
flank = 10000

conditions = ['A', 'B', 'C']

cooler_paths = {
    'A' : 'PATH_TO/A.cool',
    'B' : 'PATH_TO/B.cool',
    'C' : 'PATH_TO/C.cool',
}

long_names = {
    'A': 'long_name_A',
    'B': 'long_name_B',
    'C': 'long_name_C',
}

In [ ]:
# add fall colormap

def listToColormap(colorList, cmapName=None):
    colorList = np.array(colorList)
    if colorList.min() < 0:
        raise ValueError("Colors should be 0 to 1, or 0 to 255")
    if colorList.max() > 1.:
        if colorList.max() > 255:
            raise ValueError("Colors should be 0 to 1 or 0 to 255")
        else:
            colorList = colorList / 255.
    return mpl.colors.LinearSegmentedColormap.from_list(cmapName, colorList, 256)

fallList = [
     (255, 255, 255), (255, 255, 204),
     (255, 237, 160), (254, 217, 118),
     (254, 178, 76), (253, 141, 60),
     (252, 78, 42), (227, 26, 28),
     (189, 0, 38), (128, 0, 38), (0, 0, 0)]


def registerList(mylist, name):
    mymap = listToColormap(mylist, name)
    mymapR = listToColormap(mylist[::-1], name + "_r")
    mymapR.set_bad('white',1.)
    mymap.set_bad('white',1.)
    mpl.cm.register_cmap(name, mymap)
    mpl.cm.register_cmap(name + "_r", mymapR)

registerList(fallList, "fall")

coolwarm = mpl.cm.coolwarm
coolwarm.set_bad('white',1.)
mpl.cm.register_cmap('coolwarm', coolwarm)

In [ ]:
#load chrom sizes
chromsizes = cooler.util.read_chromsizes(
    chromsizesfile, 
    name_patterns=(r'^chr[IVX]+$', r'chrMito$'))
chromsizes
chromosomes = list(chromsizes.index)
chromosomes

In [ ]:
pal = sns.color_palette('colorblind')

clrs = {
    cond: cooler.Cooler(cooler_paths[cond]) for cond in conditions
}

## Pileup

In [ ]:
from cooltools import snipping

In [ ]:
# site of interest
sites = pd.read_table(centromeresfile)

sites.head()

In [ ]:
supports = [(chrom, 0, chromsizes[chrom]) for chrom in chromosomes]

cens = snipping.make_bin_aligned_windows(
    binsize, 
    sites['chrom'], 
    (sites['start'] + sites['end'])//2,
    flank_bp=flank)
cens['strand'] = sites['strand']
cens = snipping.assign_regions(cens, supports)
cens = cens.dropna()

print(len(cens), 'cen windows, after assigning supports')
cens.head()

In [ ]:
stacks = {}
piles = {}
for cond in conditions:
    snipper = snipping.CoolerSnipper(clrs[cond])
    
    stack = snipping.pileup(cens, snipper.select, snipper.snip)
    
    # mirror reflect snippets whose feature is on the opposite strand
    mask = np.array(cens.strand == '-', dtype=bool)
    stack[:, :, mask] = stack[::-1, ::-1, mask]
    
    stacks[cond] = stack
    piles[cond] = np.nanmean(stack, axis=2)

In [ ]:
gs = GridSpec(nrows=1, ncols=len(conditions) + 1, width_ratios=[20] * len(conditions) + [1])
plt.figure(figsize=(5 * len(conditions), 5))

opts = dict(
    vmin=-3,
    vmax=-1,
    extent=[-flank//1000, flank//1000, -flank//1000, flank//1000],
    cmap='fall'
)

for i, cond in enumerate(conditions):
    ax = plt.subplot(gs[i])
    img = ax.matshow(
        np.log10(piles[cond]),
        **opts)
    ax.xaxis.tick_bottom()
    if i > 0:
        ax.yaxis.set_visible(False)
    plt.title(long_names[cond])

ax = plt.subplot(gs[len(conditions)])
plt.colorbar(img, cax=ax)

In [ ]:
for i, cond in enumerate(conditions):

    plt.figure(figsize=(8, 8))
    plt.imshow(
        np.log10(piles[cond]),
        **opts)

    plt.title('{}_Cens'.format(long_names[cond]))
    plt.colorbar()
    plt.savefig('{}.flank{}kb.bin{}kb.log10.Obs.Cen_Pileup.png'.format(long_names[cond], flank//1000, binsize//1000))
    